# Activations functions.

> Activations functions.  Set of act_fn.

Activation functions, forked from https://github.com/rwightman/pytorch-image-models/timm/models/layers/activations.py

Mish: Self Regularized  
Non-Monotonic Activation Function  
https://github.com/digantamisra98/Mish  
fastai forum discussion https://forums.fast.ai/t/meet-mish-new-activation-function-possible-successor-to-relu  


Mish is in Pytorch from version 1.9. Use this version!  

In [1]:
# hide
# forked from https://github.com/rwightman/pytorch-image-models/timm/models/layers/activations.py

import torch
from torch import nn as nn
from torch.nn import functional as F

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [2]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

## Mish, MishJit, MishJitMe - memory-efficient.

In [3]:
from model_constructor.activations import mish, Mish, MishJit, MishMe

In [4]:
#hide_input
print_doc(mish)
print_doc(Mish)
print_doc(MishJit)
print_doc(MishMe)

<h4 id="mish" class="doc_header"><code>mish</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/activations.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>mish</code>(**`x`**, **`inplace`**:`bool`=*`False`*)

Mish: A Self Regularized Non-Monotonic Neural Activation Function - https://arxiv.org/abs/1908.08681
NOTE: I don't have a working inplace variant

<h4 id="Mish" class="doc_header"><code>class</code> <code>Mish</code><a href="torch/nn/modules/activation.py#L401" class="source_link" style="float:right">[source]</a></h4>

> <code>Mish</code>(**`inplace`**:`bool`=*`False`*) :: `Module`

Applies the Mish function, element-wise.
Mish: A Self Regularized Non-Monotonic Neural Activation Function.

.. math::
    \text{Mish}(x) = x * \text{Tanh}(\text{Softplus}(x))

.. note::
    See `Mish: A Self Regularized Non-Monotonic Neural Activation Function <https://arxiv.org/abs/1908.08681>`_

Shape:
    - Input: :math:`(N, *)` where `*` means, any number of additional
      dimensions
    - Output: :math:`(N, *)`, same shape as the input

Examples::

    >>> m = nn.Mish()
    >>> input = torch.randn(2)
    >>> output = m(input)

<h4 id="MishJit" class="doc_header"><code>class</code> <code>MishJit</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/activations.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>MishJit</code>(**`inplace`**:`bool`=*`False`*) :: `Module`

Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super(Model, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

:ivar training: Boolean represents whether this module is in training or
                evaluation mode.
:vartype training: bool

<h4 id="MishMe" class="doc_header"><code>class</code> <code>MishMe</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/activations.py#L81" class="source_link" style="float:right">[source]</a></h4>

> <code>MishMe</code>(**`inplace`**:`bool`=*`False`*) :: `Module`

Mish: A Self Regularized Non-Monotonic Neural Activation Function - https://arxiv.org/abs/1908.08681
A memory efficient, jit scripted variant of Mish

## HardMishJit, HardMishJitMe - memory efficient.

In [5]:
from model_constructor.activations import HardMishJit, HardMishMe

In [6]:
#hide_input
print_doc(HardMishJit)
print_doc(HardMishMe)

<h4 id="HardMishJit" class="doc_header"><code>class</code> <code>HardMishJit</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/activations.py#L100" class="source_link" style="float:right">[source]</a></h4>

> <code>HardMishJit</code>(**`inplace`**:`bool`=*`False`*) :: `Module`

Hard Mish
Experimental, based on notes by Mish author Diganta Misra at
  https://github.com/digantamisra98/H-Mish/blob/0da20d4bc58e696b6803f2523c58d3c8a82782d0/README.md

<h4 id="HardMishMe" class="doc_header"><code>class</code> <code>HardMishMe</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/activations.py#L144" class="source_link" style="float:right">[source]</a></h4>

> <code>HardMishMe</code>(**`inplace`**:`bool`=*`False`*) :: `Module`

A memory efficient, jit scripted variant of Hard Mish
Experimental, based on notes by Mish author Diganta Misra at
  https://github.com/digantamisra98/H-Mish/blob/0da20d4bc58e696b6803f2523c58d3c8a82782d0/README.md

# end
model_constructor
by ayasyrev